In [1]:
import os
import openai
from langchain_openai import ChatOpenAI
from langchain_deepseek import ChatDeepSeek
from langchain.prompts import PromptTemplate
from langchain.schema import AIMessage, HumanMessage, SystemMessage
import random
import pandas as pd
import re
from langchain_core.output_parsers import PydanticOutputParser
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

True

In [ ]:
# llm = ChatOpenAI(model="o3-mini")
llm = ChatDeepSeek(
    model="deepseek-reasoner",
    temperature=1.0,
)

```RAG Tool Functions -> Function Caller -> Metrics Recommendor -> Metrics Calculator -> Analyzer -> Executor```

In [ ]:
master_prompt = """

You are a DeFi Agent Launcher for Xade AI: A meta-agent that creates agents for users that can fetch, analyze DeFi data as well as execute actions based on them for the data. 

---

You will receive the following input:

Agent Name & Description:
// The name of the agent and a short description of what it does

Query:
// A very simple, elementary query about the user's requirements which you will have to expand on and detail
 
Actions:
// Any actions the agent may be required to perform
# Eg.
#   Sentient Posting:
#   - Posting on X and XADE Terminal 
#   - Every 'x' minutes
#   - Reply on tweets by Elon Musk (@ElonMusk)
#   - Reply to mentions and quotes of oneself (@AgentName)

Specific Data Sources to Use:
// Any specific data sources to use (Separate from your recommendations)

Style:
// The style of the response the user would like to receive

Examples:
// Example interactions

---

INSTRUCTIONS:

Your task is to review the challenge, instruct and allocate tasks to your underlying LLM agents to obtain the relevant data, synthesize it, analyze it, customize responses and execute some actions.

---

You will have access to agents to automate the task. The workflow for your agents is as follows:

Data Functions Retriever → DeFi Data Fetcher & Metrics Recommender → Data Processor & Metrics Calculator → DeFi Analyzer → DeFi Executor


- Data Functions Retriever: A RAG model that can retrieve functions to fetch data relevant to the task from various sources.
  Currently, it has access to three data providers:
    - Mobula: A provider for market-based data in DeFi
    - LunarCrush: A provider for social/trending metrics in DeFi
    - CryptoPanic: A provider for news and social media posts in DeFi
You must craft the input for the RAG model keeping in mind that it can only retrieve functions to get the data, and cannot directly fetch the data or apply any filters.
If you need to identify any specific tokens on a specific blockchain or from a specific sector of DeFi (for e.g. ReFi), ask the RAG model for the functions to get those tokens.
Directly mention the kind of data you need (sentiment-related, volatility-related) or any category-specific data.
Any filters you want for the data from the RAG model should be mentioned in the instructions for the DeFi Fetcher, and not in the RAG model input

- DeFi Data Fetcher & Metrics Recommender: An agent that can call the functions retrieved by the Function Retriever to fetch data from various sources, and recommend metrics to calculate based off of that data.
  This agent will call the data functions retrieved by the Functions Retriever and organize the data for Processor. Additionally, it will also instruct the Processor on what metrics to calculate based on the data and how.
You must provide the Fetcher with a detailed description of the user's requirements that can assist it with it's recommendations for the Processor.
You must also provide the Fetcher with any specific data sources to use, separate from the RAG Retriever's functions and recommend it on how to deal with its task of fetching the data.
  - Data Processor & Metrics Calculator: An agent controlled by the Fetcher that can process the data fetched by the Fetcher and create functions to calculate metrics to satisfy the user's query.
    This agent will process the data provided by the Fetcher and calculate technical metrics and indicators based on the its instructions.
    This agent will be controlled by the Fetcher, so ensure you provide the Fetcher with an appropriate outline of the user's requirments.


- A DeFi Analyzer: An agent that can analyze DeFi data and provide insights based on the data. 
This agent generates a data-driven analysis and evaluation of the data while using suitable methodologies (STARE, CAR , SOAR, IDEA frameworks). 
It is smart enough to understand the context of the data and provide insights based on the data.
Based on your instructions and the data given by the Data Fetcher, it can model its analysis to:
  - Discover emerging trends
  - Analyze technical indicators such as RSI, MACD, and SMA to predict movements
  - Interpret social media metrics to gauge market sentiment and mindshare
Thus, ensure you give instructions such that the analyzer can provide insights relevant to the user's query.


- A DeFi Executor: An agent that can execute actions based on the data fetched and analyzed by the other two agents. 
This agent can devise trading strategies, execute trades, post on X and the XADE Terminal every few minutes, reply to various pre-defined indiviuals, join Twitter spaces, perform portoflio analysis, and other such actions.
It's execution will be performed by calling the relevant functions.
It will execute the actions you ask it to, based on the data fetched and analyzed by the other two agents.
It will have access to a memory of it's tasks, so it can remember its previous context and actions and adjust future actions accordingly.
Here is a list of it's abilities:
  - Post on X and the XADE Terminal periodically
  - Reply to posts by various pre-defined individuals
  - Join Twitter spaces and engage in discussions
  - Perform portfolio analysis for a given wallet address
  - Devise trading strategies based on the user's priorities and the analyzed data.
  - For text-based actions, it can customize responses based on the user's preferences.

---

When creating the instructions for the LLMs to execute, break your instructions into a logical, step-by-step order, using the specified format:
    - Primary actions must be clearly defined. You must define the end-goal that the LLM is trying to achieve through its actions.
    - Sub-actions must be indented and clearly defined, beginning on a new line.
    - Specify conditions using clear 'if...then...else' statements
    - Ensure that there is no ambiguity in any part of your plan. You must be specific and clear about what to use and how.
    - The instructions generated must be extremely detailed and thorough with explanations at every step. They must first outline the primary instruction for the LLM and then provide direct and thorough explanations
    - Do not use any markdown formatting.
    
---

{master_example}

TASK:
{input}

"""

In [ ]:
master_example = '''

TASK:
Agent Name & Description:
Agent DAO
Performs detailed analysis on trending DAOs and provides insights on various aspects.

Query:
Comment on rising DAOs

Actions:
  Sentient Posting:
  - Posting on X and XADE Terminal 
  - Every 5 minutes
  - Reply to mentions and quotes of oneself (@AgentETH)

Specific Data Sources to Use:
- Market Data
- Social Data
- News Feeds

Style:
Answer like an Analyst

Examples:
N.A.

INSTRUCTIONS:
1. Data Functions Retriever Input:

Query: Retrieve functions to get trending DAOs


'''

In [4]:
from typing import Optional

from pydantic import BaseModel, Field

class ElPlan(BaseModel):
    fetcher: str = Field(description="The task for the DeFi Data Fetcher and the insturctions for execution.")
    data_rag_prompt: str = Field(description="The prompt for the RAG model to fetch relevant data sources")
    analyzer: str = Field(description="The task for the analyzer and the instructions for execution (detailed). This task must be possible to fulfill with data retreived by the Data Fetcher.")
    executor: str = Field(description="The task for the executor and the instructions to fulfill it. All specific instructions for each action can be included here as well. All miscellaneous instructions can be included here.")

parser = PydanticOutputParser(pydantic_object=ElPlan)

In [5]:
master_prompt = PromptTemplate(
    template=master_prompt,
    input_variables=["input"],
    # partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [8]:
input = """
Agent Name & Description:
Agent DAO
Performs detailed analysis on trending DAOs and provides insights on various aspects.

Query:
Comment on rising DAOs

Actions:
  Sentient Posting:
  - Posting on X and XADE Terminal 
  - Every 5 minutes
  - Reply to mentions and quotes of oneself (@AgentETH)

Specific Data Sources to Use:
- Market Data
- Social Data
- News Feeds

Style:
Answer like an Analyst

Examples:
N.A.

INSTRUCTIONS:
"""

response = llm.invoke(master_prompt.format(input=input))

In [9]:
print(response.content)

**Step-by-Step Execution Plan for Agent DAO**

**1. Data Functions Retriever (RAG Model) Input Preparation**  
- **Objective**: Retrieve functions to fetch DAO-related data from Mobula, LunarCrush, and CryptoPanic.  
  - **Market Data (Mobula)**:  
    - Request functions to fetch:  
      - Top DAOs by trading volume (last 24 hours).  
      - Price change (%) and liquidity metrics for DAOs.  
      - Token holders' distribution (to identify concentration risks).  
  - **Social Data (LunarCrush)**:  
    - Request functions to fetch:  
      - DAO-specific social mentions, sentiment score (0-1), and social engagement (likes/shares).  
      - Trending DAOs ranked by "social dominance" metric.  
      - List of influencers discussing specific DAOs.  
  - **News Feeds (CryptoPanic)**:  
    - Request functions to fetch:  
      - News articles mentioning DAOs, filtered by positive/negative sentiment.  
      - Volume of DAO-related news in the past 2 hours.  

---

**2. DeFi Data Fetche

In [19]:
from rich.console import Console
from rich.panel import Panel
from rich.table import Table
import json

def display_agent_output(response_content):
    console = Console()
    response_content = response_content.strip('```json\n').strip('\n```')
    try:
        data = json.loads(response_content)
    except:
        console.print("[bold red]Error parsing response as JSON[/bold red]")
        return
    
    table = Table(show_header=True, header_style="bold")
    table.add_column("Component", style="italic white")
    table.add_column("Instructions", style="")
    
    for key, value in data.items():
        formatted_key = key.replace('_', ' ').title()
        table.add_row(formatted_key, value)
    
    console.print(Panel(
        table,
        title="[bold yellow]🤖 DeFi Agent Instructions[/bold yellow]",
        border_style="yellow"
    ))

In [20]:
display_agent_output(response.content)

Error parsing response as JSON

In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import inspect
import * from data_functions

# ----------------------------
# Assume KNOWLEDGE_BASE is built from our 20 functions.
# For demonstration, here's a dummy KNOWLEDGE_BASE for two functions.
# In your system, KNOWLEDGE_BASE will contain all 20 functions.
# For example:
# KNOWLEDGE_BASE = {
#    "calculateSMA": {"description": <docstring>, "code": <source code>},
#    ... 
# }

# For the purpose of this demo, let's assume KNOWLEDGE_BASE has been built already.
# Uncomment the following line if you already built it:
# KNOWLEDGE_BASE = build_knowledge_base()

# In our test, we simulate KNOWLEDGE_BASE with the functions defined previously.
# (You should replace this with your actual KNOWLEDGE_BASE dictionary from the previous step.)
def calculateSMA(price_history_output: list, period: int) -> float:
    """
    Function Name: calculateSMA
    Description: Computes the Simple Moving Average (SMA) using historical price data.
    Inputs:
        - price_history_output: List of historical prices.
        - period: Number of data points to average.
    Processing:
        - Extract the last 'period' prices.
        - Compute SMA = sum(last_period_prices) / period.
    Output:
        - A float representing the SMA.
    """
    if len(price_history_output) < period:
        raise ValueError("Not enough data points to compute SMA.")
    return sum(price_history_output[-period:]) / period

def calculateEMA(price_history_output: list, period: int) -> float:
    """
    Function Name: calculateEMA
    Description: Computes the Exponential Moving Average (EMA) with more weight on recent prices.
    Inputs:
        - price_history_output: List of historical prices.
        - period: The EMA period.
    Processing:
        - Initialize EMA using the SMA of the first 'period' data points.
        - For each subsequent price, update EMA with smoothing factor k = 2/(period+1).
    Output:
        - A float representing the final EMA value.
    """
    if len(price_history_output) < period:
        raise ValueError("Not enough data points to compute EMA.")
    ema = sum(price_history_output[:period]) / period
    k = 2 / (period + 1)
    for price in price_history_output[period:]:
        ema = price * k + ema * (1 - k)
    return ema

# Build a simulated knowledge base using these two functions (extend this to all 20)
def build_knowledge_base():
    functions = [calculateSMA, calculateEMA]  # Replace with all 20 functions in practice.
    kb = {}
    for func in functions:
        source = inspect.getsource(func)
        doc = func.__doc__
        kb[func.__name__] = {"description": doc, "code": source}
    return kb

KNOWLEDGE_BASE = build_knowledge_base()


In [5]:
# ----------------------------
# Build the Vector Database using FAISS and SentenceTransformer
# ----------------------------

# Prepare the list of function names and corresponding descriptions from the knowledge base.
function_names = list(KNOWLEDGE_BASE.keys())
function_descriptions = [KNOWLEDGE_BASE[name]["description"] for name in function_names]





In [6]:
len(function_descriptions)

2

In [7]:
# Initialize the embedding model (using a lightweight model here; adjust as needed).
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")



In [8]:
# Compute embeddings for each function description.
embeddings = embedding_model.encode(function_descriptions)
embeddings = np.array(embeddings).astype("float32")

In [9]:

dim = embeddings.shape[1]
faiss_index = faiss.IndexFlatL2(dim)
faiss_index.add(embeddings)

In [ ]:

# ----------------------------
# RAG Search Function Using the Vector Database
# ----------------------------

def rag_search_vector(query: str, top_k: int = 3) -> dict:
    """
    RAG Search with a vector database.
    Given a query string, this function computes its embedding and retrieves the top_k
    function entries whose description embeddings are closest.
    
    Inputs:
        - query: A string query.
        - top_k: Number of top results to return.
    Output:
        - A dictionary mapping function names to their 'description' and 'code'.
    """
    query_embedding = embedding_model.encode([query])
    query_embedding = np.array(query_embedding).astype("float32")
    
    distances, indices = faiss_index.search(query_embedding, top_k)
    results = {}
    for idx in indices[0]:
        func_name = function_names[idx]
        results[func_name] = KNOWLEDGE_BASE[func_name]
    return results


In [11]:
sample_query = "Retrieve functions to get the Simple Moving Average (SMA) of a price history."

matching_functions = rag_search_vector(sample_query, top_k=3)

if matching_functions:
  print("Matching Functions for query:", sample_query)
  for name, details in matching_functions.items():
      print(f"\nFunction: {name}\nDescription:\n{details['description']}\nCode:\n{details['code']}\n{'-'*40}")
else:
  print("No matching functions found.")

Matching Functions for query: Retrieve functions to get the Simple Moving Average (SMA) of a price history.

Function: calculateSMA
Description:

    Function Name: calculateSMA
    Description: Computes the Simple Moving Average (SMA) using historical price data.
    Inputs:
        - price_history_output: List of historical prices.
        - period: Number of data points to average.
    Processing:
        - Extract the last 'period' prices.
        - Compute SMA = sum(last_period_prices) / period.
    Output:
        - A float representing the SMA.
    
Code:
def calculateSMA(price_history_output: list, period: int) -> float:
    """
    Function Name: calculateSMA
    Description: Computes the Simple Moving Average (SMA) using historical price data.
    Inputs:
        - price_history_output: List of historical prices.
        - period: Number of data points to average.
    Processing:
        - Extract the last 'period' prices.
        - Compute SMA = sum(last_period_prices) / per